In [118]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


links = [("Arab","https://www.transfermarkt.com/saudi-professional-league/transfers/wettbewerb/SA1", "23/24"),
         ("UK","https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1", "23/24"),
         ("Spain", "https://www.transfermarkt.com/laliga/transfers/wettbewerb/ES1", "23/24"),
         ("Germany", "https://www.transfermarkt.com/bundesliga/transfers/wettbewerb/L1", "23/24"),
         ("France", "https://www.transfermarkt.com/ligue-1/transfers/wettbewerb/FR1", "23/24"),
         ("France","https://www.transfermarkt.com/ligue-1/transfers/wettbewerb/FR1/plus/?saison_id=2023&s_w=&leihe=1&intern=0&intern=1", "22/23")]

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

dfs =[]
for league,link, season in links:
    res = requests.get(link, headers=headers)
    clubs=[]
    soup = BeautifulSoup(res.text)
    for h2 in soup.find_all("h2", {"class": "content-box-headline content-box-headline--inverted content-box-headline--logo"}):
        clubs.append(h2.find("a")['title'])
    list_of_df=pd.read_html(res.content)
    list_of_df=list_of_df[1:]
    for i,df in  enumerate(list_of_df):
        if df.columns[0] == "In":
            df["type_of_transfer"] = "In"
            df=df.rename(columns={"In":"Name", "Left.1":"Old_or_new_club"})
            df= df.drop(columns=["Left"])
        else:
            df["type_of_transfer"] = "Out"
            df=df.rename(columns={"Out":"Name", "Joined.1":"Old_or_new_club"})
            df= df.drop(columns=["Joined"])
        df["Club"] = clubs[i//2]
        df["league"] = league
        df["season"]= season
        dfs.append(df)


df = pd.concat(dfs)

In [119]:
def clean_values(df, column_name):
    df[column_name]=df[column_name].str.replace("€","")
    df[column_name]=df[column_name].str.replace("Loan fee: ","")
    new_values=[]
    for val in df[column_name].values:
        if val.count("m")!=0:
            val = val.replace("m","")
            val = float(val)
            val*= 1_000_000
        elif val.count("k")!=0:
            val = val.replace("k","")
            val = float(val)
            val*= 1_000
        else:
            val = -1.0
        new_values.append(val)
    df.drop(columns=[column_name],inplace=True)
    return new_values

    
df["Market_value"]=clean_values(df, "Market value")
df["Fee"] = clean_values(df, "Fee")


In [120]:
df

,Name,Age,Nat.,Position,Pos,Old_or_new_club,type_of_transfer,Club,league,season,Market_value,Fee
0,Riyad MahrezR. Mahrez,32,NaN,Right Winger,RW,Man City,In,Al-Ahli SFC,Arab,23/24,18000000.0,35000000.0
1,Gabri VeigaG. Veiga,21,NaN,Central Midfield,CM,Celta de Vigo,In,Al-Ahli SFC,Arab,23/24,30000000.0,30000000.0
2,Roger IbañezRoger Ibañez,24,NaN,Centre-Back,CB,AS Roma,In,Al-Ahli SFC,Arab,23/24,28000000.0,30000000.0
3,Allan Saint-MaximinA. Saint-Maximin,26,NaN,Left Winger,LW,Newcastle,In,Al-Ahli SFC,Arab,23/24,24000000.0,27200000.0
4,Merih DemiralM. Demiral,25,NaN,Centre-Back,CB,Atalanta BC,In,Al-Ahli SFC,Arab,23/24,20000000.0,20000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8,Lamine GueyeL. Gueye,25,NaN,Right Winger,RW,Paris FC,Out,FC Metz,France,22/23,800000.0,-1.0
9,Sofiane AlakouchS. Alakouch,25,NaN,Right-Back,RB,Paris FC,Out,FC Metz,France,22/23,600000.0,-1.0
10,VagnerVagner,27,NaN,Right Winger,RW,Without Club,Out,FC Metz,France,22/23,600000.0,-1.0
11,Papa Ndiaga YadeP. Yade,23,NaN,Left Winger,LW,QRM,Out,FC Metz,France,22/23,1000000.0,-1.0


In [114]:
df2=df[(df["Market_value"]!=-1) & (df["Fee"]!=-1)]

In [97]:
ins=df2[df2["type_of_transfer"]=="In"].groupby(["Club","league"]).sum(["Market_value","Fee"])[["Market_value","Fee"]].reset_index()
outs =df2[df2["type_of_transfer"]=="Out"].groupby(["Club","league"]).sum(["Market_value","Fee"])[["Market_value","Fee"]].reset_index()


In [98]:
ins

,Club,league,Market_value,Fee
0,1.FC Heidenheim 1846,Germany,5000000.0,2300000.0
1,1.FC Köln,Germany,7500000.0,2500000.0
2,1.FC Union Berlin,Germany,47500000.0,32000000.0
3,1.FSV Mainz 05,Germany,10500000.0,9000000.0
4,AFC Bournemouth,UK,125000000.0,127690000.0
...,...,...,...,...
87,VfL Bochum,Germany,6500000.0,2450000.0
88,VfL Wolfsburg,Germany,87500000.0,72500000.0
89,Villarreal CF,Spain,18000000.0,12500000.0
90,West Ham United,UK,129500000.0,137560000.0


In [100]:
outs

,Club,league,Market_value,Fee
0,1.FC Köln,Germany,4000000.0,2700000.0
1,1.FC Union Berlin,Germany,6500000.0,2100000.0
2,1.FSV Mainz 05,Germany,32700000.0,23550000.0
3,AFC Bournemouth,UK,2200000.0,1500000.0
4,AS Monaco,France,57300000.0,59000000.0
...,...,...,...,...
72,VfB Stuttgart,Germany,43300000.0,49900000.0
73,VfL Wolfsburg,Germany,71500000.0,72100000.0
74,Villarreal CF,Spain,153500000.0,109500000.0
75,West Ham United,UK,146000000.0,156400000.0


In [117]:
import plotly.express as px

fig = px.scatter(df2, x="Market_value", y="Fee", color="league", trendline="ols", hover_name="Name")
fig.show()

In [108]:
res=ins.groupby("league").sum()[["Fee","Market_value"]].reset_index()
res["overpay"] = res["Fee"] - res["Market_value"]

In [111]:
fig = px.bar(res, x="league", y="overpay", color="league")
fig.show()